# Imports

In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import requests
import time
import numpy_financial as npf
from pandas_datareader import data, wb
import pandas_datareader.data as web

In [2]:
pd.options.display.max_rows = 999

### Price

In [3]:
now = datetime.datetime.now()
start_year = int(now.strftime('%Y'))-5
end_year = int(now.strftime('%Y'))
day = int(now.strftime('%d'))
month = int(now.strftime('%m'))
start = datetime.datetime(start_year, month, day)
end = datetime.datetime(end_year, month, day)

def data_pull(tick):
    df = web.DataReader(f'{tick}', 'yahoo', start, end)
    df['Company']= f'{tick}'
    return df

In [4]:
stock_pricing=data_pull('aapl')

In [5]:
stock_pricing

,High,Low,Open,Close,Volume,Adj Close,Company
Date,,,,,,,
2015-08-11,118.180000,113.330002,117.809998,113.489998,97082800.0,104.877724,aapl
2015-08-12,115.419998,109.629997,112.529999,115.239998,101217500.0,106.494896,aapl
2015-08-13,116.400002,114.540001,116.040001,115.150002,48535800.0,106.411743,aapl
2015-08-14,116.309998,114.010002,114.320000,115.959999,42929500.0,107.160263,aapl
2015-08-17,117.650002,115.500000,116.040001,117.160004,40884700.0,108.269211,aapl
...,...,...,...,...,...,...,...
2020-08-05,441.570007,435.589996,437.510010,440.250000,30498000.0,439.457642,aapl
2020-08-06,457.649994,439.190002,441.619995,455.609985,50607200.0,454.790009,aapl
2020-08-07,454.700012,441.170013,452.820007,444.450012,49453300.0,444.450012,aapl


In [6]:
stock_price = stock_pricing['Close'][-1]

In [7]:
ticker='aapl'

### Outstanding Shares

In [83]:
outstanding_shares = stock_pricing['Volume'][-1]

In [84]:
outstanding_shares

45109051.0

### Total Dividend Payouts

In [85]:
cfs = pd.read_html(f'https://www.marketwatch.com/investing/stock/{ticker}/financials/cash-flow')
cash_flow = pd.DataFrame(cfs[2]) #[1 or 2]
cash_flow = cash_flow.transpose()
new_header = cash_flow.iloc[0]
cash_flow = cash_flow[1:]
cash_flow.columns = new_header
totdiv = cash_flow['Cash Dividends Paid - Total'][4]
totdiv=totdiv.replace('.', '')
totdiv=totdiv.replace('B','0000000')
totdiv=totdiv.replace('(','')
totdiv=totdiv.replace(')','')
totdiv=float(totdiv)

In [86]:
totdiv

14120000000.0

In [12]:
# this number is 2019. The others are 2020. May have to find a way to do trailing 12 months



### Net Income

In [13]:
inc_st = pd.read_html(f'https://www.marketwatch.com/investing/stock/{ticker}/financials')
income_state = pd.DataFrame(inc_st[1])
income_state = income_state.transpose()
new_header = income_state.iloc[0]
income_state = income_state[1:]
income_state.columns = new_header
net_income =  income_state['Net Income'][4]
net_income=net_income.replace('.', '')
net_income=net_income.replace('B','0000000')
net_income=float(net_income)

In [14]:
net_income

55260000000.0

### Annual Div Per Share

In [15]:
DPS = totdiv/outstanding_shares

In [87]:
DPS

145.44285908523446

### DataFrame

In [17]:
divdata = {'OutstandingShares':  [outstanding_shares],
        'PPS': [stock_price],
        'TotalDividends': [totdiv],
        'NetIncome':[net_income]  
        }



In [18]:
df = pd.DataFrame(divdata)

In [19]:
df

,OutstandingShares,PPS,TotalDividends,NetIncome
0,97082800.0,437.5,1.412000e+10,5.526000e+10


In [20]:
dividend_yield_ratio=DPS/stock_price

In [21]:
dividend_yield_ratio

0.3324408207662502

In [88]:
dividend_payout_ratio=totdiv/net_income

In [90]:
dividend_payout_ratio

0.2555193630112197

------------------------------------------------------------------------

### Historical Dividend DataFrame

In [24]:
ticker='aapl'

In [25]:
histdivs=pd.read_html(f'https://finance.yahoo.com/quote/{ticker}/history?period1=1438214400&period2=1596067200&interval=div%7Csplit&filter=div&frequency=1mo')

In [26]:
histdivs=histdivs[0]

In [27]:
histdivs.drop(histdivs.tail(1).index,inplace=True)

In [28]:
histdivs.drop(columns= ['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'], inplace=True)

In [29]:
histdivs['Dividends']= histdivs['Dividends'].apply(lambda x: x.replace(' Dividend', ''))

In [30]:
histdivs['Dividends']=histdivs['Dividends'].astype(float)

In [31]:
histdivs

,Date,Dividends
0,"May 08, 2020",0.82
1,"Feb 07, 2020",0.77
2,"Nov 07, 2019",0.77
3,"Aug 09, 2019",0.77
4,"May 10, 2019",0.77
5,"Feb 08, 2019",0.73
6,"Nov 08, 2018",0.73
7,"Aug 10, 2018",0.73
8,"May 11, 2018",0.73
9,"Feb 09, 2018",0.63


In [32]:
histdivs['Date'] =  pd.to_datetime(histdivs['Date'])

In [33]:
histdivs

,Date,Dividends
0,2020-05-08,0.82
1,2020-02-07,0.77
2,2019-11-07,0.77
3,2019-08-09,0.77
4,2019-05-10,0.77
5,2019-02-08,0.73
6,2018-11-08,0.73
7,2018-08-10,0.73
8,2018-05-11,0.73
9,2018-02-09,0.63


In [34]:
def historical_dividends(ticker):
    histdivs=pd.read_html(f'https://finance.yahoo.com/quote/{ticker}/history?period1=1438214400&period2=1596067200&interval=div%7Csplit&filter=div&frequency=1mo')
    histdivs=histdivs[0]
    histdivs.drop(histdivs.tail(1).index,inplace=True)
    histdivs.drop(columns= ['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'], inplace=True)
    histdivs['Dividends']= histdivs['Dividends'].apply(lambda x: x.replace(' Dividend', ''))
    histdivs['Dividends']=histdivs['Dividends'].astype(float)
    histdivs['Date'] =  pd.to_datetime(histdivs['Date'])
    return histdivs

In [35]:
historical_dividends('aapl')

,Date,Dividends
0,2020-05-08,0.82
1,2020-02-07,0.77
2,2019-11-07,0.77
3,2019-08-09,0.77
4,2019-05-10,0.77
5,2019-02-08,0.73
6,2018-11-08,0.73
7,2018-08-10,0.73
8,2018-05-11,0.73
9,2018-02-09,0.63


# Dividend Yield Ratio Function

In [114]:
def dividend_yield(ticker):
    divs=historical_dividends(f'{ticker}')
    curdiv=divs['Dividends'][0]
    numerator=curdiv*4
    prices=data_pull(f'{ticker}')
    price_now=prices['Close'][-1]
    dyr=numerator/price_now*100
    return dyr
    

In [115]:
dividend_yield('aapl')

0.7497142857142857

# Dividend Payout Ratio Function

In [208]:
inc_st = pd.read_html('https://www.marketwatch.com/investing/stock/aapl/financials')
income_state = pd.DataFrame(inc_st[1])
income_state = income_state.transpose()

In [209]:
new_header = income_state.iloc[0]
income_state = income_state[1:]
income_state.columns = new_header

In [210]:
eps_df = income_state['EPS (Basic)']
eps_df = eps_df.to_frame()
eps_df.reset_index(inplace=True)
eps_df.drop([0,1,2,3,5],inplace=True)
eps_df.rename(columns={"index": "date", "EPS (Basic)": "eps"}, inplace=True)
eps_df.reset_index(inplace=True, drop=True)

In [212]:
use_year=int(eps_df['date'][0])

In [213]:
eps=float(eps_df['eps'][0])

In [214]:
divs=historical_dividends('aapl')

In [216]:
divs_use_year = divs[divs['Date'].dt.year == use_year]

In [218]:
numerator=divs_use_year['Dividends'].sum()

In [219]:
dpr=(numerator/eps)*100

In [220]:
dpr

25.396825396825395

In [225]:
def dividend_payout(ticker):
    inc_st = pd.read_html(f'https://www.marketwatch.com/investing/stock/{ticker}/financials')
    income_state = pd.DataFrame(inc_st[1])
    income_state = income_state.transpose()
    new_header = income_state.iloc[0]
    income_state = income_state[1:]
    income_state.columns = new_header
    eps_df = income_state['EPS (Basic)']
    eps_df = eps_df.to_frame()
    eps_df.reset_index(inplace=True)
    eps_df.drop([0,1,2,3,5],inplace=True)
    eps_df.rename(columns={"index": "date", "EPS (Basic)": "eps"}, inplace=True)
    eps_df.reset_index(inplace=True, drop=True)
    use_year=int(eps_df['date'][0])
    eps=float(eps_df['eps'][0])
    divs=historical_dividends(f'{ticker}')
    divs_use_year = divs[divs['Date'].dt.year == use_year]
    numerator=divs_use_year['Dividends'].sum()
    dpr=(numerator/eps)*100
    return dpr

In [226]:
dividend_payout('aapl')

25.396825396825395

###################

### Adjusting Stock price info

In [36]:
pricer=pd.read_html(F'https://finance.yahoo.com/quote/{ticker}/history?period1=1438214400&period2=1596067200&interval=1mo&filter=history&frequency=1mo')

In [37]:
pricer=pricer[0]
pricer.drop(pricer.tail(1).index,inplace=True)
pricer.drop(columns= ['Open', 'High','Low','Adj Close**'], inplace=True)
pricer['Date'] =  pd.to_datetime(pricer['Date'])

In [38]:
pricer

,Date,Close*,Volume
0,2020-07-01,425.04,755124000
1,2020-06-01,364.80,810739300
2,2020-05-08,0.82 Dividend,0.82 Dividend
3,2020-05-01,317.94,701520600
4,2020-04-01,293.80,816222400
5,2020-03-01,254.29,1570018100
6,2020-02-07,0.77 Dividend,0.77 Dividend
7,2020-02-01,273.36,754584100
8,2020-01-01,309.51,733592600
9,2019-12-01,293.65,597198700


In [39]:
droplist = [2,6,10,14,18,22,26,30,34,38,42,46,50,54,58,62,66,70,74,78]

In [40]:
pricer.drop(index=droplist, inplace=True)

In [41]:
pricer

,Date,Close*,Volume
0,2020-07-01,425.04,755124000
1,2020-06-01,364.80,810739300
3,2020-05-01,317.94,701520600
4,2020-04-01,293.80,816222400
5,2020-03-01,254.29,1570018100
7,2020-02-01,273.36,754584100
8,2020-01-01,309.51,733592600
9,2019-12-01,293.65,597198700
11,2019-11-01,267.25,448331500
12,2019-10-01,248.76,608302700


In [42]:
def historical_price(ticker):
    pricer=pd.read_html(f'https://finance.yahoo.com/quote/{ticker}/
                        history?period1=1438214400&period2=1596067200&
                        interval=1mo&filter=history&frequency=1mo')
    pricer=pricer[0]
    pricer.drop(pricer.tail(1).index,inplace=True)
    pricer.drop(columns= ['Open', 'High','Low','Adj Close**'], inplace=True)
    pricer['Date'] =  pd.to_datetime(pricer['Date'])
    droplist = [2,6,10,14,18,22,26,30,34,38,42,46,50,54,58,62,66,70,74,78]
    pricer.drop(index=droplist, inplace=True)
    return pricer

In [43]:
historical_price(ticker)

,Date,Close*,Volume
0,2020-07-01,425.04,755124000
1,2020-06-01,364.80,810739300
3,2020-05-01,317.94,701520600
4,2020-04-01,293.80,816222400
5,2020-03-01,254.29,1570018100
7,2020-02-01,273.36,754584100
8,2020-01-01,309.51,733592600
9,2019-12-01,293.65,597198700
11,2019-11-01,267.25,448331500
12,2019-10-01,248.76,608302700


### Historics Merging

In [44]:
historics= pd.merge(pricer, histdivs, how='outer')

In [45]:
historics

,Date,Close*,Volume,Dividends
0,2020-07-01,425.04,755124000,NaN
1,2020-06-01,364.80,810739300,NaN
2,2020-05-01,317.94,701520600,NaN
3,2020-04-01,293.80,816222400,NaN
4,2020-03-01,254.29,1570018100,NaN
5,2020-02-01,273.36,754584100,NaN
6,2020-01-01,309.51,733592600,NaN
7,2019-12-01,293.65,597198700,NaN
8,2019-11-01,267.25,448331500,NaN
9,2019-10-01,248.76,608302700,NaN


In [46]:
historics=historics.sort_values(by='Date',ascending=False)

In [47]:
cols = list(historics.columns.values)

In [48]:
cols

['Date', 'Close*', 'Volume', 'Dividends']

In [49]:
column_titles = ['Date','Close*','Volume','Dividends']

historics=historics.reindex(columns=column_titles)

In [50]:
historics

,Date,Close*,Volume,Dividends
0,2020-07-01,425.04,755124000,NaN
1,2020-06-01,364.80,810739300,NaN
60,2020-05-08,NaN,NaN,0.82
2,2020-05-01,317.94,701520600,NaN
3,2020-04-01,293.80,816222400,NaN
4,2020-03-01,254.29,1570018100,NaN
61,2020-02-07,NaN,NaN,0.77
5,2020-02-01,273.36,754584100,NaN
6,2020-01-01,309.51,733592600,NaN
7,2019-12-01,293.65,597198700,NaN


In [51]:
historics.reset_index(drop=True, inplace=True)

In [52]:
historics['Ticker']= ticker

In [53]:
historics

,Date,Close*,Volume,Dividends,Ticker
0,2020-07-01,425.04,755124000,NaN,aapl
1,2020-06-01,364.80,810739300,NaN,aapl
2,2020-05-08,NaN,NaN,0.82,aapl
3,2020-05-01,317.94,701520600,NaN,aapl
4,2020-04-01,293.80,816222400,NaN,aapl
5,2020-03-01,254.29,1570018100,NaN,aapl
6,2020-02-07,NaN,NaN,0.77,aapl
7,2020-02-01,273.36,754584100,NaN,aapl
8,2020-01-01,309.51,733592600,NaN,aapl
9,2019-12-01,293.65,597198700,NaN,aapl


In [81]:
def stock_history(ticker):
### this function runs two functions to compile history stock price 
### and dividend data. It then compiles it into one dataframe
    divy=historical_dividends(ticker)
    pricy=historical_price(ticker)
    historics= pd.merge(pricy, divy, how='outer')
    historics=historics.sort_values(by='Date',ascending=False)
    cols = list(historics.columns.values)
    column_titles = ['Date','Close*','Volume','Dividends']
    historics=historics.reindex(columns=column_titles)
    historics.reset_index(drop=True, inplace=True)
    historics['Ticker']= ticker
    historics.set_index('Date', inplace=True)
    # Below is growth rate and outputs
    divgrowthrate=(divy['Dividends'].iloc[0]-divy['Dividends'].iloc[-1])/divy['Dividends'].iloc[0]*100
    print (f'Historical Stock Price & Dividends for {ticker.upper()}')
    print ('----------------------------------------------------------')
    print (f'Five Year Dividend Growth Rate: {divgrowthrate}%')
    return historics

In [82]:
stock_history(ticker)

Historical Stock Price & Dividends for AAPL
----------------------------------------------------------
Five Year Dividend Growth Rate 36.58536585365853%


,Close*,Volume,Dividends,Ticker
Date,,,,
2020-07-01,425.04,755124000,NaN,aapl
2020-06-01,364.80,810739300,NaN,aapl
2020-05-08,NaN,NaN,0.82,aapl
2020-05-01,317.94,701520600,NaN,aapl
2020-04-01,293.80,816222400,NaN,aapl
2020-03-01,254.29,1570018100,NaN,aapl
2020-02-07,NaN,NaN,0.77,aapl
2020-02-01,273.36,754584100,NaN,aapl
2020-01-01,309.51,733592600,NaN,aapl


# Dividend Analysis Function

In [234]:
def analyze_dividend_performance(ticker):
    ### this function runs multiple functions to analyze stock dividend performance
    try:
        divy=historical_dividends(ticker)
        pricy=historical_price(ticker)
        historics= pd.merge(pricy, divy, how='outer')
        historics=historics.sort_values(by='Date',ascending=False)
        cols = list(historics.columns.values)
        column_titles = ['Date','Close*','Volume','Dividends']
        historics=historics.reindex(columns=column_titles)
        historics.reset_index(drop=True, inplace=True)
        historics['Ticker']= ticker
        historics.set_index('Date', inplace=True)
        # Below is growth rate and outputs
        divgrowthrate=(divy['Dividends'].iloc[0]-divy['Dividends'].iloc[-1])/divy['Dividends'].iloc[0]*100
        dyr=dividend_yield(f'{ticker}')
        dpr=dividend_payout(f'{ticker}')
        print (f'Dividend Performance for {ticker.upper()}')
        print ('----------------------------------------------------------')
        print (f'Five Year Dividend Growth Rate: {divgrowthrate}%')
        print (f'Dividend Yield Ratio: {dyr}%')
        print (f'Dividend Payout Ratio: {dpr}%')
        ########
        return historics
    except:
        print (f'Could not analyze {ticker.upper()}. Confirm stock ticker is correct and the stock pays dividends.')

In [236]:
analyze_dividend_performance('msft')

Dividend Performance for MSFT
----------------------------------------------------------
Five Year Dividend Growth Rate: 39.21568627450981%
Dividend Yield Ratio: 1.6127445772704818%
Dividend Payout Ratio: 17.525773195876287%


,Close*,Volume,Dividends,Ticker
Date,,,,
2020-07-01,205.01,770510200,NaN,msft
2020-06-01,203.51,764905600,NaN,msft
2020-05-20,NaN,NaN,0.51,msft
2020-05-01,183.25,688854900,NaN,msft
2020-04-01,179.21,984596200,NaN,msft
2020-03-01,157.71,1612695500,NaN,msft
2020-02-19,NaN,NaN,0.51,msft
2020-02-01,162.01,887402700,NaN,msft
2020-01-01,170.23,558530000,NaN,msft
